# Machine Vision 2024/25 - Assignment 3: Line Fitting

Consider the image `res/postit2g.png` from the previous assignment. Where we already identified edge pixels
using canny edge detection and parametric lines using the hough transform.

In this assignment we will continue fitting parametric lines using different approaches.

In [ ]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import skimage.io
import skimage.color

#### Part 1: Line Fitting using Total Least Squares

##### Preparation

First, visualize the image and understand its shape

In [ ]:
# @student: visualize image and print its shape
gray_img = ...
print(f"gray_img shape [v,u]: {...}")
...

The image is a grayscale image, hence it has only one color dimension/channel.
However, we want to visualize the found lines in red.
Therefore, we need to convert it to RGB color space

In [ ]:
color_img = skimage.color.gray2rgb(gray_img)
print(f"color_img shape [v,u,c]: {color_img.shape}")
plt.imshow(color_img)
plt.show()

The file `res/edge_pixels.npy` contains 19 lists, where each list represents a dominant edge.

In [ ]:
lists_of_edgepixels = np.load("res/edge_pixels.npy", allow_pickle=True)

# Each list contains of a different amount of pixels, where each pixel consists of an u and v coordinate
print(f"pixels in first list: {lists_of_edgepixels[0].shape}")
print(f"pixels in second list: {lists_of_edgepixels[1].shape}")

# @student: List the u pixels and the v pixels of the third list of edge pixels
# In this list the index of u coordinate is 0 and the index of the coordinate v=1
u3 = ...
v3 = ...

# copy color_img and change the color of edge pixels to red
# it is important to use copy.deepcopy to copy the contents of the array (image)
color_img_edge3 = copy.deepcopy(color_img)

# @student: change the color of u0 and v0 to red
color_img_edge3[...] = ...
plt.imshow(color_img_edge3)
plt.show()

##### Calculation

Using a list of edgepixels we can fit a line through these using the total least squares aproach.

In [ ]:
def fit_line_total_least_squares(edgepixels):
    # @student: Calculate alpha, beta, gamma
    u = ...
    N = ...

    u_sum = ...
    v_sum = ...
    u_sum_2 = ...
    v_sum_2 = ...
    uv_sum = ...
    alpha = ...
    beta = ...
    gamma = ...

    # Build up matrix M and calculate its smallest eigenvector (var: evec)
    M = np.ndarray(shape=(2,2))
    M[0,0] = alpha
    M[0,1] = beta
    M[1,0] = beta
    M[1,1] = gamma
    evals, evecs = np.linalg.eig(M) # calculate eigenvalues / eigenvectors
    eigen_sorted = np.argsort(evals) # sort eigenvalues
    evec = evecs[:, eigen_sorted[0]] # get eigenvector with the smallest eigenvalue

    # @student: Calculate c
    c = ...

    return evec, c

In [ ]:
# for each list of edge pixels we create an array containing the line parameters
# Therefore, each entry consists of (0: n0, 1: n1, 2: c)
lines = np.ndarray(shape=(len(lists_of_edgepixels), 3))
print(f"lines shape: {lines.shape}")
for idx, edgepixels in enumerate(lists_of_edgepixels):
    n, c = fit_line_total_least_squares(edgepixels)
    lines[idx] = [n[0], n[1], c]

In order to visualize the line we create an array with values ranging from `0` to `max(u)`
Afterwards, we apply the line equation for each u in this range

In [ ]:
# @student: create span for u
u = ...
for line in lines:
    # @student: apply line equation to calculate v based on the line parameters and u
    v = ...

    random_color = np.random.rand(3,)
    plt.plot(u, v, c=random_color)

plt.imshow(color_img)
plt.xlim(0, color_img.shape[1]) # limit u axis to image dimension
plt.ylim(color_img.shape[0], 0) # limit v axis to image dimension
plt.show()

We successfully applied total least squares in order to estimate line parameters from a list of pixels.
However, we don't know the start and end of the line within the image yet.
In order to do so, we project each edge pixel to the computed line and take the first and last projected point as start and end.

In [ ]:
def calculate_start_end(*, line, edgepixels):
    # @student: project edgepixels to the line
    d = ...
    projection = ...

    min_idx, _ = np.argmin(projection, axis=1)
    max_idx, _ = np.argmax(projection, axis=1)
    return [projection[0,min_idx], projection[0,max_idx]], [projection[1,min_idx], projection[1,max_idx]]

for idx, edgepixels in enumerate(lists_of_edgepixels):
    line = lines[idx]
    line_start_u, line_end_v = calculate_start_end(line=line, edgepixels=edgepixels)
    random_color = np.random.rand(3,)
    plt.plot(line_start_u, line_end_v, c=random_color, marker="o", mfc='none')
plt.imshow(color_img)
plt.show()

#### Part 2: Line Fitting using RANSAC

In comparison to total least squares, RANSAC is robust against outliers. In this exercise you will implement RANSAC and compare it to total least squares.

In [ ]:
def find_two_random_pixels(*, edgepixels):
    # @student: find two different random pixels within the edgepixels array

    return ...

In [ ]:
def calculate_nc(*, p1, p2):
    # @student: based on two pixels p1 and p2 calculate the parameters n,c (line in normal form) between p1 and p2
    ...
    n = ...
    c = ...

    return n, c

In [ ]:
def calculate_num_inliers(*, edgepixels, n, c, threshold):
    # @student: Calculate the number on inliers
    return ...

In [ ]:
def fit_line_ransac(*, edgepixels, k, threshold):
    best_num_inliers = 0
    best_n = None
    best_c = None
    best_p1 = None
    best_p2 = None

    # iterate k times
    for _ in range(k):
        # Generate two random pixels from the edgepixels
        p1, p2 = find_two_random_pixels(edgepixels=edgepixels)

        # Calculate n , c
        n, c = calculate_nc(p1=p1, p2=p2)

        # Calculate how many points are inliers within the threshold t based on the distance to the line
        num_inliers = calculate_num_inliers(edgepixels=edgepixels, n=n, c=c, threshold=threshold)

        # If the line has more inliers, it is considered as "better" solution
        if num_inliers > best_num_inliers:
            best_num_inliers = num_inliers
            best_n = n
            best_c = c
            best_p1 = p1
            best_p2 = p2

    return best_n, best_c, best_p1, best_p2

In [ ]:
u = np.arange(0, color_img.shape[1], 1)
for idx, edgepixels in enumerate(lists_of_edgepixels):
    n, c, p1, p2 = fit_line_ransac(edgepixels=edgepixels, k=100, threshold=1)

    # @student: apply line equation to calculate v based on parameters and u
    v = ...

    random_color = np.random.rand(3,)
    plt.imshow(color_img)
    plt.plot(u, v, c=random_color)
    plt.scatter([p1[0], p2[0]],[p1[1], p2[1]], edgecolors=random_color, marker="o", facecolors="none")
plt.xlim(0, color_img.shape[1])
plt.ylim(color_img.shape[0], 0)
plt.show()

Take the union of the pixels belonging to the first and second pixellist. Apply both, the least-sum-of-squares estimator
and the ransac algorithm to the joint list. In which way do the results differ?

In [ ]:
# @student: concatenate the first two lists
u12 = ...
v12 = ...
composed_edgepixels = ...

# @student: copy a new color img and display edge 1 and 2 in red
...

In [ ]:
# @student: apply total least squares and visualize the result
...

In [ ]:
# @student: apply the ransac algorithm and visualize the result
...